## 读取数据

In [6]:
# 首先 import 必要的模块
import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import GridSearchCV
#打开文件
# path to where the data lies
dpath = './input/'
train = pd.read_csv(dpath +'train_data_org1.csv',index_col='用户编码')
y_train=train.pop('信用分')
X_train=train.drop(['data_type'], axis=1)

In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
def score(y_pred, y_true):
    return 1/(mean_absolute_error(y_pred, y_true)+1)
My_Score = make_scorer(score, greater_is_better=True)

## LGBMRegressor

In [8]:
params = {
    'boosting_type': 'gbdt', 
    'objective': 'regression_l2', 
     'metric': 'mae',
    'learning_rate': 0.01, 
    'num_leaves': 50, 
    'max_depth': 7,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'subsample': 0.8, 
    'lambda_l1': 0,
    'lambda_l2': 5, 
    'nthread': 4
    }
data_train = lgb.Dataset(X_train, y_train, silent=True)
cv_results = lgb.cv(
    params, data_train, num_boost_round=10000, nfold=5, stratified=False, shuffle=True, metrics='mae',
    early_stopping_rounds=150, verbose_eval=50, show_stdv=True, seed=0)

print('best n_estimators:', len(cv_results['l1-mean']))
print('best cv score:', cv_results['l1-mean'][-1])

[50]	cv_agg's l1: 24.9438 + 0.173121
[100]	cv_agg's l1: 20.0876 + 0.13607
[150]	cv_agg's l1: 17.5674 + 0.0970465
[200]	cv_agg's l1: 16.3182 + 0.0783871
[250]	cv_agg's l1: 15.6578 + 0.0764052
[300]	cv_agg's l1: 15.343 + 0.0810653
[350]	cv_agg's l1: 15.1594 + 0.0853833
[400]	cv_agg's l1: 15.0556 + 0.0887548
[450]	cv_agg's l1: 14.9876 + 0.0928272
[500]	cv_agg's l1: 14.9456 + 0.0955622
[550]	cv_agg's l1: 14.918 + 0.0960883
[600]	cv_agg's l1: 14.8981 + 0.0964241
[650]	cv_agg's l1: 14.8836 + 0.0978782
[700]	cv_agg's l1: 14.8708 + 0.0997128
[750]	cv_agg's l1: 14.8618 + 0.100701
[800]	cv_agg's l1: 14.8544 + 0.101576
[850]	cv_agg's l1: 14.8486 + 0.102076
[900]	cv_agg's l1: 14.8456 + 0.10267
[950]	cv_agg's l1: 14.8426 + 0.102379
[1000]	cv_agg's l1: 14.841 + 0.102254
[1050]	cv_agg's l1: 14.8386 + 0.101944
[1100]	cv_agg's l1: 14.8371 + 0.101482
[1150]	cv_agg's l1: 14.836 + 0.10151
[1200]	cv_agg's l1: 14.8355 + 0.101084
[1250]	cv_agg's l1: 14.8339 + 0.102296
[1300]	cv_agg's l1: 14.833 + 0.102137
[1

In [36]:
from sklearn.model_selection import GridSearchCV
### 我们可以创建lgb的sklearn模型，使用上面选择的(学习率，评估器数目)
params_test1={
    'max_depth': [5],
    'num_leaves':[35,40]
}
model_lgb=lgb.LGBMRegressor(objective='regression_l1',num_leaves=50,bagging_fraction=1,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=3068, max_depth=6,feature_fraction=0.8,
                              metric='mae', bagging_freq = 5,  min_child_samples=20,min_child_weight=0.00001)
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, scoring=My_Score, cv=5, verbose=3, n_jobs=4)
gsearch1.fit(X_train, y_train)
evalute_result = gsearch1.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(gsearch1.best_params_))
print('最佳模型得分:{0}'.format(gsearch1.best_score_))

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:  4.7min remaining:  2.0min
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  5.8min finished


每轮迭代运行结果:[0.0635995 0.0635995]
参数的最佳取值：{'max_depth': 5, 'num_leaves': 35}
最佳模型得分:0.06359950418743296


In [5]:
params_test3={
    'min_child_samples': [16,17],
    'min_child_weight':[0.00001]
}
model_lgb=lgb.LGBMRegressor(objective='regression_l1',num_leaves=50,bagging_fraction=1,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=3068, max_depth=6,feature_fraction=0.8,
                              metric='mae', bagging_freq = 5,  min_child_samples=20,min_child_weight=0.00001)
gsearch3 = GridSearchCV(estimator=model_lgb, param_grid=params_test3, scoring=My_Score, cv=5, verbose=1, n_jobs=4)
gsearch3.fit(X_train, y_train)
evalute_result = gsearch3.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(gsearch3.best_params_))
print('最佳模型得分:{0}'.format(gsearch3.best_score_))

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  6.8min finished


每轮迭代运行结果:[0.06360584 0.06362892]
参数的最佳取值：{'min_child_samples': 17, 'min_child_weight': 1e-05}
最佳模型得分:0.06362891574734529


In [8]:
cv_params={
    'feature_fraction': [0.4,0.5,  0.6],
    'bagging_fraction': [0.5,0.6,  0.7]
}
model_lgb = lgb.LGBMRegressor(objective='regression_l1',num_leaves=50,bagging_fraction=1,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=3068, max_depth=6,feature_fraction=0.8,
                              metric='mae', bagging_freq = 5,  min_child_samples=17,min_child_weight=0.00001)
Grid = GridSearchCV(estimator=model_lgb, param_grid=cv_params, scoring=My_Score, cv=5, verbose=1, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed: 30.1min finished


每轮迭代运行结果:[0.06381896 0.06382503 0.06380691 0.06380721 0.06382926 0.06378818
 0.06381483 0.06379999 0.06382521]
参数的最佳取值：{'bagging_fraction': 0.6, 'feature_fraction': 0.5}
最佳模型得分:0.06382925777757613


In [ ]:
cv_params={
    'reg_alpha': [0.0005,0.001,0.002],
    'reg_lambda': [0.0005,0.001,0.002]
}
model_lgb = lgb.LGBMRegressor(objective='regression_l1',num_leaves=50,bagging_fraction=0.6,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=3068, max_depth=6,feature_fraction=0.5,
                              metric='mae', bagging_freq = 5,  min_child_samples=17,min_child_weight=0.00001)
Grid = GridSearchCV(estimator=model_lgb, param_grid=cv_params, scoring=My_Score, cv=5, verbose=1, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [9]:
cv_params={
    'learning_rate': [0.001, 0.1,0.2]
}
model_lgb = lgb.LGBMRegressor(objective='regression_l1',num_leaves=50,bagging_fraction=0.6,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=3068, max_depth=6,feature_fraction=0.5,
                              metric='mae', bagging_freq = 5,  min_child_samples=17,min_child_weight=0.00001)
Grid = GridSearchCV(estimator=model_lgb, param_grid=cv_params, scoring=My_Score, cv=5, verbose=1, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed:   17.1s finished


每轮迭代运行结果:[0.06321324 0.06329426 0.06271503]
参数的最佳取值：{'learning_rate': 0.1}
最佳模型得分:0.06329426174166188


## 测试集

In [3]:
from lightgbm.sklearn import LGBMRegressor
model = lgb.LGBMRegressor(objective='regression_l1',num_leaves=50,bagging_fraction=0.6,reg_alpha=0.001, reg_lambda=0.001,
                              learning_rate=0.01, n_estimators=3068, max_depth=6,feature_fraction=0.5,
                              metric='mae', bagging_freq = 5,  min_child_samples=17,min_child_weight=0.00001)
model.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.6, bagging_freq=5, boosting_type='gbdt',
       class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
       importance_type='split', learning_rate=0.01, max_depth=6,
       metric='mae', min_child_samples=17, min_child_weight=1e-05,
       min_split_gain=0.0, n_estimators=3068, n_jobs=-1, num_leaves=50,
       objective='regression_l1', random_state=None, reg_alpha=0.001,
       reg_lambda=0.001, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [4]:
#打开文件
# path to where the data lies
dpath = './input/'
test = pd.read_csv(dpath +'test_data_org.csv',index_col='用户编码')
X_test=test.drop(['data_type'], axis=1)
X_test_index=X_test.index
#y预测，四舍五入，取整
y_pred=np.around(model.predict(X_test)).astype(int)
#生成提交结果
user_id=pd.DataFrame(X_test_index).rename(columns = {'用户编码':'id'})
score = pd.DataFrame(y_pred,columns=['score'])
out_df = pd.concat([user_id,score], axis = 1)
out_df.to_csv("./output/LGBM.csv", index=False)